## Follow the cup

In [1]:
from jetbot import ObjectDetector

model = ObjectDetector('ssd_mobilenet_v2_coco.engine')

In [2]:
from jetbot import Camera
from jetbot import bgr8_to_jpeg

camera = Camera.instance(width=300, height=300)

In [18]:
from IPython.display import display
import ipywidgets.widgets as widgets

image_widget = widgets.Image(format='jpeg', width=300, height=300)
label_widget = widgets.IntText(value=47, description='tracked label')
cx_widget = widgets.FloatText(value=0, description='cx')
cy_widget = widgets.FloatText(value=0, description='cy')
tolerance_widget = widgets.FloatSlider(description='tolerance', min=0.05, max=0.5, value=0.10, step=0.01)
speed_widget = widgets.FloatSlider(description='speed', min=0.05, max=0.5, value=0.10, step=0.01)

display(image_widget, label_widget, cx_widget, cy_widget, tolerance_widget, speed_widget)

Image(value=b'', format='jpeg', height='300', width='300')

IntText(value=47, description='tracked label')

FloatText(value=0.0, description='cx')

FloatText(value=0.0, description='cy')

FloatSlider(value=0.1, description='tolerance', max=0.5, min=0.05, step=0.01)

FloatSlider(value=0.1, description='speed', max=0.5, min=0.05, step=0.01)

In [19]:
from jetbot import Robot

robot = Robot()

In [20]:
import json

descriptions=[]

with open('mscoco_complete_label_map.json') as json_file:
    data = json.load(json_file)
    for p in data['items']:
        descriptions.append(p['display_name'])


In [21]:
import cv2
import numpy as np

width = int(image_widget.width)
height = int(image_widget.height)

def drawBoundingBox(imgcv,box,label,color):
    x1,y1,x2,y2 = (int(width * box[0]), int(height * box[1]), int(width * box[2]), int(height * box[3]))
    cv2.rectangle(imgcv,(x1,y1),(x2,y2),color,2)
    labelSize=cv2.getTextSize(label,cv2.FONT_HERSHEY_COMPLEX,0.5,2)
    _x1 = x1
    _y1 = y1
    _x2 = x1+labelSize[0][0]
    _y2 = y1-int(labelSize[0][1])
    cv2.rectangle(imgcv,(_x1,_y1),(_x2,_y2),color,cv2.FILLED)
    cv2.putText(imgcv,label,(x1,y1),cv2.FONT_HERSHEY_COMPLEX,0.5,(255,255,255),1)
    return imgcv
    
def detection_center(detection):
    """Computes the center x, y coordinates of the object"""
    bbox = detection['bbox']
    center_x = (bbox[0] + bbox[2]) / 2.0 - 0.5
    center_y = (bbox[1] + bbox[3]) / 2.0 - 0.5
    return (center_x, center_y)
    
def norm(vec):
    """Computes the length of the 2D vector"""
    return np.sqrt(vec[0]**2 + vec[1]**2)

def closest_detection(detections):
    """Finds the detection closest to the image center"""
    closest_detection = None
    for det in detections:
        center = detection_center(det)
        if closest_detection is None:
            closest_detection = det
        elif norm(detection_center(det)) < norm(detection_center(closest_detection)):
            closest_detection = det
    return closest_detection

def execute(change):
    image = change['new']
    
    # compute all detected objects
    detections = model(image)
    
    # draw all detections on image
    
    for det in detections[0]:
        image=drawBoundingBox(image,det['bbox'],descriptions[det['label']],(255,0,0))

    # select detections that match selected class label
    matching_detections = [d for d in detections[0] if d['label'] == int(label_widget.value)]
    
    # get detection closest to center of field of view and draw it
    det = closest_detection(matching_detections)
    if det is not None:
        image=drawBoundingBox(image,det['bbox'],descriptions[det['label']],(0,0,255))
        center = detection_center(det)
        cx_widget.value=center[0]
        cy_widget.value=center[1]
        
        limit=float(tolerance_widget.value)
        speed=float(speed_widget.value)

        # turn the robot
        if center[0] > limit:
            robot.set_motors( speed,-speed)
        elif center[0] < -limit:
            robot.set_motors(-speed, speed)
        else:
            robot.stop()
    else:
        robot.stop()
    
    # update image widget
    image_widget.value = bgr8_to_jpeg(image)
    
execute({'new': camera.value})

In [22]:
camera.unobserve_all()
camera.observe(execute, names='value')

In [15]:
camera.unobserve_all()
robot.stop()